In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, ReLU, MaxPooling2D, Flatten, Dense
from tensorflow.keras import Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.model_selection import train_test_split
import cv2

# Load images from the specified folder
def load_images_from_folder(folder_path, img_size=(28, 28)):
    """Loads images from the given folder and labels them based on folder names."""
    images = []
    labels = []
    class_names = os.listdir(folder_path)  # List folder names as class names
    for class_id, class_name in enumerate(class_names):
        class_folder = os.path.join(folder_path, class_name)
        if os.path.isdir(class_folder):  # Process only subdirectories (classes)
            for filename in os.listdir(class_folder):
                img_path = os.path.join(class_folder, filename)
                img = load_img(img_path, target_size=img_size, color_mode="grayscale")
                img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
                images.append(img_array)
                labels.append(class_id)
    return np.array(images), np.array(labels), class_names

# Define the Prototypical Network
class ProtoNet(Model):
    def __init__(self):
        super(ProtoNet, self).__init__()
        self.conv1 = Conv2D(64, (3, 3), padding='same')
        self.bn1 = BatchNormalization()
        self.pool1 = MaxPooling2D((2, 2))
        self.flatten = Flatten()
        self.fc = Dense(64, activation='relu')

    def call(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = ReLU()(x)
        x = self.pool1(x)
        x = self.flatten(x)
        return self.fc(x)

# Compute prototypes
def compute_prototypes(model, x_support, y_support, num_classes):
    """Compute class prototypes."""
    embeddings = model(x_support)
    prototypes = []
    for c in range(num_classes):
        prototypes.append(tf.reduce_mean(embeddings[y_support == c], axis=0))
    return tf.stack(prototypes)

# Classify query images
def classify_query_images(model, x_query, prototypes):
    """Classify query images based on distance from nearest prototype."""
    query_embeddings = model(x_query)
    distances = tf.norm(tf.expand_dims(query_embeddings, axis=1) - prototypes, axis=2)
    return tf.argmin(distances, axis=1)

# Function to upload and preprocess a new image
def upload_and_preprocess_image(image_path, target_size=(28, 28)):
    """Loads and preprocesses the uploaded image."""
    img = load_img(image_path, target_size=target_size, color_mode="grayscale")
    img_array = img_to_array(img) / 255.0  # Normalize to [0, 1]
    img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Main script
if __name__ == "__main__":
    # Specify the dataset path directly in the code
    dataset_path = '/content/drive/MyDrive/FewShot'  # Replace with your folder path
    print(f"Dataset loaded from: {dataset_path}")

    # Define parameters
    img_size = (28, 28)
    shots_per_class = 20
    query_per_class = 10

    # Load dataset
    x_data, y_data, class_names = load_images_from_folder(dataset_path, img_size)
    num_classes = len(class_names)

    # Split dataset into support and query sets (simulate few-shot)
    x_support, x_query, y_support, y_query = train_test_split(
        x_data, y_data, test_size=query_per_class / len(x_data), random_state=42
    )

    # Instantiate the model
    model = ProtoNet()

    # Train the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    epochs = 5
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            support_embeddings = model(x_support)
            query_embeddings = model(x_query)

            prototypes = compute_prototypes(model, x_support, y_support, num_classes)

            distances = tf.norm(tf.expand_dims(query_embeddings, axis=1) - prototypes, axis=2)
            predicted = tf.argmin(distances, axis=1)

            loss = tf.keras.losses.sparse_categorical_crossentropy(y_query, distances)
            acc = tf.reduce_mean(tf.cast(tf.equal(predicted, y_query), tf.float32))

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(f"Epoch {epoch + 1}/{epochs} - Loss: {tf.reduce_mean(loss):.4f}, Accuracy: {acc:.4f}")

    # Test the model
    prototypes = compute_prototypes(model, x_support, y_support, num_classes)
    predictions = classify_query_images(model, x_query, prototypes)
    print("Class Names:", class_names)

    # Now, classify an uploaded image
    image_path = '/content/drive/MyDrive/FewShot/Cyclone/29.jpg'  # Replace with the path to your image
    uploaded_image = upload_and_preprocess_image(image_path, target_size=img_size)

    # Compute prototypes based on the support set
    prototypes = compute_prototypes(model, x_support, y_support, num_classes)

    # Classify the uploaded image
    image_embedding = model(uploaded_image)
    distances = tf.norm(tf.expand_dims(image_embedding, axis=1) - prototypes, axis=2)
    predicted_class = tf.argmin(distances, axis=1).numpy()[0]

    print(f"Uploaded image is classified as: {class_names[predicted_class]}")


Dataset loaded from: /content/drive/MyDrive/FewShot
Epoch 1/5 - Loss: 0.7705, Accuracy: 0.7000
Epoch 2/5 - Loss: 0.8141, Accuracy: 0.8000
Epoch 3/5 - Loss: 0.6710, Accuracy: 0.7000
Epoch 4/5 - Loss: 0.6681, Accuracy: 0.7000
Epoch 5/5 - Loss: 0.6509, Accuracy: 0.7000
Class Names: ['Cyclone', 'Wildfire']
Uploaded image is classified as: Cyclone


In [3]:
# Main script
if __name__ == "__main__":
    # Specify the dataset path directly in the code
    dataset_path = '/content/drive/MyDrive/FewShot'  # Replace with your folder path
    print(f"Dataset loaded from: {dataset_path}")

    # Define parameters
    img_size = (28, 28)
    shots_per_class = 20
    query_per_class = 10

    # Load dataset
    x_data, y_data, class_names = load_images_from_folder(dataset_path, img_size)
    num_classes = len(class_names)

    # Split dataset into support and query sets (simulate few-shot)
    x_support, x_query, y_support, y_query = train_test_split(
        x_data, y_data, test_size=query_per_class / len(x_data), random_state=42
    )

    # Instantiate the model
    model = ProtoNet()

    # Train the model
    optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)
    epochs = 3

    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            support_embeddings = model(x_support)
            query_embeddings = model(x_query)

            prototypes = compute_prototypes(model, x_support, y_support, num_classes)

            distances = tf.norm(tf.expand_dims(query_embeddings, axis=1) - prototypes, axis=2)
            predicted = tf.argmin(distances, axis=1)

            loss = tf.keras.losses.sparse_categorical_crossentropy(y_query, distances)
            acc = tf.reduce_mean(tf.cast(tf.equal(predicted, y_query), tf.float32))

        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        print(f"Epoch {epoch + 1}/{epochs} - Loss: {tf.reduce_mean(loss):.4f}, Accuracy: {acc:.4f}")

    # Test the model
    prototypes = compute_prototypes(model, x_support, y_support, num_classes)
    predictions = classify_query_images(model, x_query, prototypes)

    print("Class Names:", class_names)

    # Now, classify an uploaded image
    image_path = '/content/drive/MyDrive/FewShot/Wildfire/24.jpg'  # Replace with the path to your image
    uploaded_image = upload_and_preprocess_image(image_path, target_size=img_size)

    # Compute prototypes based on the support set
    prototypes = compute_prototypes(model, x_support, y_support, num_classes)

    # Classify the uploaded image
    image_embedding = model(uploaded_image)
    distances = tf.norm(tf.expand_dims(image_embedding, axis=1) - prototypes, axis=2)
    predicted_class = tf.argmin(distances, axis=1).numpy()[0]

    print(f"Uploaded image is classified as: {class_names[predicted_class]}")


Dataset loaded from: /content/drive/MyDrive/FewShot
Epoch 1/3 - Loss: 0.7573, Accuracy: 0.8000
Epoch 2/3 - Loss: 0.8613, Accuracy: 0.8000
Epoch 3/3 - Loss: 0.6831, Accuracy: 0.7000
Class Names: ['Cyclone', 'Wildfire']
Uploaded image is classified as: Wildfire
